# potential_LAMMPS record generator

This Notebook contains functions to help generate potential_LAMMPS records for different pair styles.  

**NOTE** Methods should be generalized and better integrated with the package later.  Maybe as classes?

In [1]:
import os
import uuid

from DataModelDict import DataModelDict as DM

## 1. Single potential parameter file styles

Included pair_styles: 

- adp
- bop
- comb, comb3
- eam/* (NOT eam!)
- gw, gw/zbl
- lcbop
- nb3b/harmonic
- polymorphic
- sw
- tersoff and tersoff/*
- vashishta and vashishta/*

### potential_lammps_file()

Parameters

- __id__ (*str*) Unique name to call the potential implementation.

- __paramfile__ (*str*) Name of the potential parameter file.

- __pot_id__ (*str, optional*) Unique name associated with the potential model concept.  If not given, will use everything in id prior to "--LAMMPS--".

- __pot_key__ (*str, optional*) UUID4 associated with the potential model concept. If not given, a new UUID4 will be generated.

- __units__ (*str, optional*) The LAMMPS units style to use with the potential.  Default value is "metal".

- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the potential.  Default value is "atomic".

- __pair_style__ (*str, optional*) The LAMMPS pair_style to use with the potential.  Default value is "eam/alloy".

- __elements__ (*list, optional*) The list of the elemental species tags associated with each atomic model in the potential.  Either elements or symbols is required.

- __masses__ (*list, optional*) The list of the atomic masses associated with each atomic model in the potential. Optional if elements is given as the standard atomic weights of the elements will be used.

- __symbols__ (*list, optional*) The list of the symbol names associated with each atomic model in the potential. Not needed if the symbol names correspond to the model's elemental species tags (which is true for most interatomic potentials of this format).  If symbols is given and elements is not, masses is required.

Returns

(*DataModelDict.DataModelDict*) The data model for the record which can then be saved in either JSON or XML format.


**NOTE** function needs to be updated for pair_style line representation, e.g. for LAMMPS command line 
    
    pair_style comb3 polar_on

there should be a function parameter to take "polar_on" keyword separately from "comb3".

In [2]:
def potential_lammps_file(id, paramfile, pot_id=None, pot_key=None,
                          units='metal', atom_style='atomic', pair_style='eam/alloy',
                          elements=[], masses=[], symbols=[]):
        
    # Initialize model
    model = DM()
    model['potential-LAMMPS'] = DM()
    
    # Add key, id
    model['potential-LAMMPS']['key'] = str(uuid.uuid4())
    model['potential-LAMMPS']['id'] = id
    
    # Add potential key, id
    model['potential-LAMMPS']['potential'] = DM()
    if pot_key is not None:
        model['potential-LAMMPS']['potential']['key'] = pot_key
    else:
        model['potential-LAMMPS']['potential']['key'] = str(uuid.uuid4())
    if pot_id is not None:
        model['potential-LAMMPS']['potential']['id'] = pot_id
    else:
        model['potential-LAMMPS']['potential']['id'] = id[:id.index('--LAMMPS--')]

    # Add LAMMPS units, atom_style
    model['potential-LAMMPS']['units'] = units
    model['potential-LAMMPS']['atom_style'] = atom_style
    
    # Handle elements, symbols, masses information
    if len(symbols) > 0:
        symbollist = ' '.join(elements)
        if len(elements) == 0:
            elements = [None for i in range(len(symbols))]    
    elif len(elements) > 0:
        symbollist = ' '.join(elements)
        symbols = [None for i in range(len(elements))]       
    else:
        raise ValueError('elements and/or symbols required')
        
    if len(masses) == 0:
        masses = [None for i in range(len(elements))]
    assert len(symbols) == len(elements), 'incompatible symbols, elements lengths'
    assert len(symbols) == len(masses), 'incompatible symbols, masses lengths'
    
    for element, symbol, mass in zip(elements, symbols, masses):
        atom = DM()
        if symbol is not None:
            atom['symbol'] = symbol
        if element is not None:
            atom['element'] = element
        if mass is not None:
            atom['mass'] = mass
        model['potential-LAMMPS'].append('atom', atom)
    
    # Add pair_style
    model['potential-LAMMPS']['pair_style'] = DM()
    model['potential-LAMMPS']['pair_style']['type'] = pair_style
    
    # Add pair_coeff
    model['potential-LAMMPS']['pair_coeff'] = DM()
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('file', paramfile)]))
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('symbols', 'True')]))
    
    return model

### Demonstration

In [3]:
model = potential_lammps_file('2018--Starikov-S-V--Si-Au--LAMMPS--ipr1',
                              paramfile='Si_Au.adp.txt',
                              pair_style='adp',
                              elements=['Si', 'Au'],
                              masses=[28.0855, 196.966],
                             )
print(model.json(indent=4))

{
    "potential-LAMMPS": {
        "key": "51d67e54-9012-4cea-a2e9-6aac9cc71dca", 
        "id": "2018--Starikov-S-V--Si-Au--LAMMPS--ipr1", 
        "potential": {
            "key": "38ae513f-c98c-4018-b9fb-68a11568974c", 
            "id": "2018--Starikov-S-V--Si-Au"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "element": "Si", 
                "mass": 28.0855
            }, 
            {
                "element": "Au", 
                "mass": 196.966
            }
        ], 
        "pair_style": {
            "type": "adp"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Si_Au.adp.txt"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


## 2. Library plus potential file styles

Included pair_styles: 

- meam
- snap

### potential_lammps_file()

Parameters

- __id__ (*str*) Unique name to call the potential implementation.

- __libfile__ (*str*) Name of the potential library file.

- __paramfile__ (*str, optional*) Name of the potential parameter file.  Optional if the libfile is the only one needed.

- __pot_id__ (*str, optional*) Unique name associated with the potential model concept.  If not given, will use everything in id prior to "--LAMMPS--".

- __pot_key__ (*str, optional*) UUID4 associated with the potential model concept. If not given, a new UUID4 will be generated.

- __units__ (*str, optional*) The LAMMPS units style to use with the potential.  Default value is "metal".

- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the potential.  Default value is "atomic".

- __pair_style__ (*str, optional*) The LAMMPS pair_style to use with the potential.  Default value is "meam".

- __elements__ (*list, optional*) The list of the elemental species tags associated with each atomic model in the potential.  Either elements or symbols is required.

- __masses__ (*list, optional*) The list of the atomic masses associated with each atomic model in the potential. Optional if elements is given as the standard atomic weights of the elements will be used.

- __symbols__ (*list, optional*) The list of the symbol names associated with each atomic model in the potential. Not needed if the symbol names correspond to the model's elemental species tags (which is true for most interatomic potentials of this format).  If symbols is given and elements is not, masses is required.

Returns

(*DataModelDict.DataModelDict*) The data model for the record which can then be saved in either JSON or XML format.

In [4]:
def potential_lammps_library(id, libfile, paramfile='NULL', pot_id=None, pot_key=None,
                            units='metal', atom_style='atomic', pair_style='meam',
                            elements=[], masses=[], symbols=[]):
        
    # Initialize model
    model = DM()
    model['potential-LAMMPS'] = DM()
    
    # Add key, id
    model['potential-LAMMPS']['key'] = str(uuid.uuid4())
    model['potential-LAMMPS']['id'] = id
    
    # Add potential key, id
    model['potential-LAMMPS']['potential'] = DM()
    if pot_key is not None:
        model['potential-LAMMPS']['potential']['key'] = pot_key
    else:
        model['potential-LAMMPS']['potential']['key'] = str(uuid.uuid4())
    if pot_id is not None:
        model['potential-LAMMPS']['potential']['id'] = pot_id
    else:
        model['potential-LAMMPS']['potential']['id'] = id[:id.index('--LAMMPS--')]

    # Add LAMMPS units, atom_style
    model['potential-LAMMPS']['units'] = units
    model['potential-LAMMPS']['atom_style'] = atom_style
    
    # Handle elements, symbols, masses information
    if len(symbols) > 0:
        symbollist = ' '.join(elements)
        if len(elements) == 0:
            elements = [None for i in range(len(symbols))]    
    elif len(elements) > 0:
        symbollist = ' '.join(elements)
        symbols = [None for i in range(len(elements))]       
    else:
        raise ValueError('elements and/or symbols required')
        
    if len(masses) == 0:
        masses = [None for i in range(len(elements))]
    assert len(symbols) == len(elements), 'incompatible symbols, elements lengths'
    assert len(symbols) == len(masses), 'incompatible symbols, masses lengths'
    
    for element, symbol, mass in zip(elements, symbols, masses):
        atom = DM()
        if symbol is not None:
            atom['symbol'] = symbol
        if element is not None:
            atom['element'] = element
        if mass is not None:
            atom['mass'] = mass
        model['potential-LAMMPS'].append('atom', atom)
    
    # Add pair_style
    model['potential-LAMMPS']['pair_style'] = DM()
    model['potential-LAMMPS']['pair_style']['type'] = pair_style
    
    # Add pair_coeff
    model['potential-LAMMPS']['pair_coeff'] = DM()
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('file', libfile)]))
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('option', symbollist)]))
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('file', paramfile)]))
    model['potential-LAMMPS']['pair_coeff'].append('term', DM([('symbols', 'True')]))
    
    return model

### Demonstration

In [5]:
model = potential_lammps_library('2018--Etesami-S-A-Asadi-E--Fe--LAMMPS--ipr1',
                                 libfile = 'library.Fe.meam',
                                 paramfile = 'Fe.meam',
                                 pair_style = 'meam',
                                 elements = ['Fe'],
                                 masses = [55.845],
                                 symbols = ['FeE'])
print(model.json(indent=4))

{
    "potential-LAMMPS": {
        "key": "2177a659-8f58-4f11-8f1d-6abd863a6641", 
        "id": "2018--Etesami-S-A-Asadi-E--Fe--LAMMPS--ipr1", 
        "potential": {
            "key": "590cf2ed-fc61-4cc4-b8d7-fe97bc965a3b", 
            "id": "2018--Etesami-S-A-Asadi-E--Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": {
            "symbol": "FeE", 
            "element": "Fe", 
            "mass": 55.845
        }, 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "library.Fe.meam"
                }, 
                {
                    "option": "Fe"
                }, 
                {
                    "file": "Fe.meam"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}
